In [ ]:
from pyAudioAnalysis.pyAudioAnalysis.audioBasicIO import read_audio_file, stereo_to_mono
from pyAudioAnalysis.pyAudioAnalysis.ShortTermFeatures import feature_extraction

signals = [stereo_to_mono(read_audio_file(f"data/{i}.mp3")[1]) for i in range(0, 8)]

In [ ]:
short_features_with_headers = [feature_extraction(s, 44100, 1000, 1000) for s in signals]

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [79]:
short_features = []
headers = []
for index, val in enumerate(short_features_with_headers):
    if index == 0:
        headers = val[1]
    short_features.append(val[0])

In [ ]:
import pandas as pd

pd.DataFrame(short_features, )

array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [4.60549040e-09, 4.60549040e-09, 4.60549040e-09, ...,
        4.60549040e-09, 4.60549040e-09, 4.60549040e-09],
       [3.32192809e+00, 3.32192809e+00, 3.32192809e+00, ...,
        3.32192809e+00, 3.32192809e+00, 3.32192809e+00],
       ...,
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]], shape=(68, 7072))

In [ ]:
distances = []

for song, sindex in short_features[1:]:
    distances.append([])
    for feature in headers:
        distances[sindex].append()